In [84]:
import re
import nltk
import numpy

In [85]:
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import LSTM

In [87]:
numpy.random.seed(7)

## Tag the training data

In [86]:
def clean_sentence(s):
    c = s.lower().strip()
    return re.sub('[^a-z ]', '', c)

In [58]:
def translate_word_class(tag):
    if tag in ['JJ', 'JJR', 'JJS']:
        return 1 #'adjective'
    if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return 2 #'noun'
    if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return 3 #'verb'
    if tag in ['CC', 'IN']:
        return 4 #'link'
    return 5 #'other'

In [9]:
def map_tagging(sentence):
    translation = list()
    for word, tag in sentence:
        translation.append((word, translate_word_class(tag)))
    return translation

In [30]:
def map_encoding(sentence):
    encoded = list()
    for word, tag in sentence:
        encodedInt = one_hot(word,300000)[0]
        encoded.append((encodedInt,tag))
    return encoded        

In [59]:
END = 6
def sentence_labeling(sentence):
    labels = list()
    for word, tag in sentence[1:]:
        labels.append(tag)
    labels.append(END)
    return labels

In [56]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [60]:
train_sentences = list()
with open('./data_set/training_set70.txt') as train:
    for line in train:
        train_sentences.append(clean_sentence(line))

In [61]:
train_sentences[:5]

['correct', 'dick agreed', 'sorry wrong', 'where is mary', 'he is happy now']

In [62]:
tagged_sentences = list()
for sentence in train_sentences:
    words = nltk.word_tokenize(sentence)
    tagged_sentences.append(nltk.pos_tag(words))

In [64]:
tagged_sentences[:5]

[[('correct', 'NN')],
 [('dick', 'NNS'), ('agreed', 'VBD')],
 [('sorry', 'NN'), ('wrong', 'JJ')],
 [('where', 'WRB'), ('is', 'VBZ'), ('mary', 'JJ')],
 [('he', 'PRP'), ('is', 'VBZ'), ('happy', 'JJ'), ('now', 'RB')]]

In [65]:
my_tagged_sentences = list(map(map_tagging, tagged_sentences))

In [66]:
my_tagged_sentences[:5]

[[('correct', 2)],
 [('dick', 2), ('agreed', 3)],
 [('sorry', 2), ('wrong', 1)],
 [('where', 5), ('is', 3), ('mary', 1)],
 [('he', 5), ('is', 3), ('happy', 1), ('now', 5)]]

In [78]:
x = flatten(map(map_encoding, my_tagged_sentences))

In [81]:
x[:5]

[(65806, 2), (253317, 2), (230520, 3), (232498, 2), (95297, 1)]

In [82]:
y = flatten(map(sentence_labeling, my_tagged_sentences))

In [83]:
y[:5]

[6, 3, 6, 1, 6]

In [92]:
model = Sequential()
model.add(LSTM(1, input_dim=2))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [93]:
model.fit(x,y,nb_epoch=3,batch_size=1)

Exception: Error when checking model input: expected lstm_input_3 to have 3 dimensions, but got array with shape (328, 2)

In [96]:
numpy.matrix(x)

matrix([[ 65806,      2],
        [253317,      2],
        [230520,      3],
        [232498,      2],
        [ 95297,      1],
        [250561,      5],
        [123789,      3],
        [ 69480,      1],
        [106348,      5],
        [123789,      3],
        [208507,      1],
        [180038,      5],
        [253317,      2],
        [ 33211,      3],
        [142580,      5],
        [138178,      2],
        [250561,      5],
        [123789,      3],
        [142679,      1],
        [225909,      2],
        [201797,      3],
        [142915,      5],
        [265412,      2],
        [252006,      5],
        [190366,      2],
        [ 83718,      5],
        [  7787,      2],
        [233862,      3],
        [ 57347,      5],
        [138178,      2],
        [  7787,      1],
        [ 52412,      3],
        [ 76173,      4],
        [142580,      5],
        [151652,      1],
        [281774,      5],
        [164770,      3],
        [ 85672,      2],
        [176